In [6]:
import cv2
import time
import datetime
from ultralytics import YOLO

# --- AGENT 1: THE OBSERVER (The Eyes) ---
class ObserverAgent:
    def __init__(self):
        # Uses YOLOv8 Nano - extremely fast for real-time
        self.model = YOLO('yolov8n.pt') 

    def get_count(self, frame):
        # Detect only class 0 (Person)
        results = self.model(frame, classes=[0], verbose=False)
        return len(results[0].boxes), results[0]

# --- AGENT 2: THE ANALYST (The Brain) ---
class AnalystAgent:
    def __init__(self, threshold=1000):
        self.threshold = threshold

    def check_status(self, current_count):
        if current_count >= self.threshold:
            return "CRITICAL"
        elif current_count >= (self.threshold * 0.8):
            return "WARNING"
        return "NORMAL"

# --- AGENT 3: THE WARDEN (The Action) ---
class WardenAgent:
    def __init__(self):
        self.log_file = "temple_security_logs.txt"

    def execute_alert(self, count):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        alert_msg = f"[{timestamp}] ALERT: Capacity Exceeded! Count: {count}"
        
        # 1. Log to File (Documentation)
        with open(self.log_file, "a") as f:
            f.write(alert_msg + "\n")
        
        # 2. Console Alert
        print(f"\033[91m {alert_msg} \033[0m") # Prints in Red text
        
        # 3. Audio Alert (Internal Beep)
        # For Linux/Mac use: os.system('say "Alert"') 
        # For Windows:
        import ctypes
        ctypes.windll.user32.MessageBeep(0xFFFFFFFF)

# --- SYSTEM INTEGRATION (End-to-End) ---

def main():
    # Initialize Agents
    observer = ObserverAgent()
    analyst = AnalystAgent(threshold=10) # Set to 10 for testing, 1000 for production
    warden = WardenAgent()

    cap = cv2.VideoCapture(0) # 0 is your default webcam

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        # STEP 1: Observer gets the count
        count, results = observer.get_count(frame)
        
        # STEP 2: Analyst checks against threshold
        status = analyst.check_status(count)

        # STEP 3: Visual Feedback and Warden Action
        color = (0, 255, 0) # Green for Normal
        
        if status == "CRITICAL":
            color = (0, 0, 255) # Red for Alert
            warden.execute_alert(count) # Warden triggers external alerts
            cv2.putText(frame, "!!! DANGER: OVERCROWDED !!!", (100, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
        
        # Draw results on screen
        annotated_frame = results.plot() 
        cv2.putText(annotated_frame, f"People in Temple: {count}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        
        # Show the video feed
        cv2.imshow("Temple Crowd Tracking System", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()